<a href="https://colab.research.google.com/github/HaojiaK/blog/blob/ML_security_and_privacy/Tensorflow_Privacy_DP_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-privacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
from absl import logging
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
from tensorflow import estimator as tf_estimator
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy_lib
from tensorflow_privacy.privacy.optimizers import dp_optimizer

tf.flags.DEFINE_boolean(
    'dpsgd', True, 'If True, train with DP-SGD. If False, '
    'train with vanilla SGD.')
tf.flags.DEFINE_float('learning_rate', .15, 'Learning rate for training')
tf.flags.DEFINE_float('noise_multiplier', 1.1,
                   'Ratio of the standard deviation to the clipping norm')
tf.flags.DEFINE_float('l2_norm_clip', 1.0, 'Clipping norm')
tf.flags.DEFINE_integer('batch_size', 256, 'Batch size')
tf.flags.DEFINE_integer('epochs', 30, 'Number of epochs')
tf.flags.DEFINE_integer(
    'microbatches', 256, 'Number of microbatches '
    '(must evenly divide batch_size)')
tf.flags.DEFINE_string('model_dir', None, 'Model directory')

FLAGS = tf.flags.FLAGS

"""common python"""
def get_cnn_model(features):
  """Given input features, returns the logits from a simple CNN model."""
  input_layer = tf.reshape(features, [-1, 28, 28, 1])
  y = tf.keras.layers.Conv2D(
      16, 8, strides=2, padding='same', activation='relu')(
          input_layer)
  y = tf.keras.layers.MaxPool2D(2, 1)(y)
  y = tf.keras.layers.Conv2D(
      32, 4, strides=2, padding='valid', activation='relu')(
          y)
  y = tf.keras.layers.MaxPool2D(2, 1)(y)
  y = tf.keras.layers.Flatten()(y)
  y = tf.keras.layers.Dense(32, activation='relu')(y)
  logits = tf.keras.layers.Dense(10)(y)

  return logits


def make_input_fn(split, input_batch_size=256, repetitions=-1, tpu=False):
  """Make input function on given MNIST split."""

  def input_fn(params=None):
    """A simple input function."""
    batch_size = params.get('batch_size', input_batch_size)

    def parser(example):
      image, label = example['image'], example['label']
      image = tf.cast(image, tf.float32)
      image /= 255.0
      label = tf.cast(label, tf.int32)
      return image, label

    dataset = tfds.load(name='mnist', split=split)
    dataset = dataset.map(parser).shuffle(60000).repeat(repetitions).batch(
        batch_size)
    # If this input function is not meant for TPUs, we can stop here.
    # Otherwise, we need to explicitly set its shape. Note that for unknown
    # reasons, returning the latter format causes performance regression
    # on non-TPUs.
    if not tpu:
      return dataset

    # Give inputs statically known shapes; needed for TPUs.
    images, labels = tf.data.make_one_shot_iterator(dataset).get_next()
    # return images, labels
    images.set_shape([batch_size, 28, 28, 1])
    labels.set_shape([
        batch_size,
    ])
    return images, labels

  return input_fn
"""end common.py"""





def cnn_model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
  """Model function for a CNN."""

  # Define CNN architecture.
  logits = get_cnn_model(features)

  # Calculate loss as a vector (to support microbatches in DP-SGD).
  vector_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits)
  # Define mean of loss across minibatch (for reporting through tf.Estimator).
  scalar_loss = tf.reduce_mean(input_tensor=vector_loss)

  # Configure the training op (for TRAIN mode).
  if mode == tf_estimator.ModeKeys.TRAIN:
    if FLAGS.dpsgd:
      # Use DP version of GradientDescentOptimizer. Other optimizers are
      # available in dp_optimizer. Most optimizers inheriting from
      # tf.compat.v1.train.Optimizer should be wrappable in differentially
      # private counterparts by calling dp_optimizer.optimizer_from_args().
      optimizer = dp_optimizer.DPGradientDescentGaussianOptimizer(
          l2_norm_clip=FLAGS.l2_norm_clip,
          #the maximum Euclidean norm of each individual gradient 
          #that is computedon an individual training examle from a minibatch
          noise_multiplier=FLAGS.noise_multiplier,
          #control how much noise is sampled and added to gradients before they 
          #are applied by the optimizer
          num_microbatches=FLAGS.microbatches,
          learning_rate=FLAGS.learning_rate)
      opt_loss = vector_loss
    else:
      optimizer = tf.compat.v1.train.GradientDescentOptimizer(
          learning_rate=FLAGS.learning_rate)
      opt_loss = scalar_loss

    global_step = tf.compat.v1.train.get_global_step()
    train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)

    # In the following, we pass the mean of the loss (scalar_loss) rather than
    # the vector_loss because tf.estimator requires a scalar loss. This is only
    # used for evaluation and debugging by tf.estimator. The actual loss being
    # minimized is opt_loss defined above and passed to optimizer.minimize().
    return tf_estimator.EstimatorSpec(
        mode=mode, loss=scalar_loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode).
  elif mode == tf_estimator.ModeKeys.EVAL:
    eval_metric_ops = {
        'accuracy':
            tf.compat.v1.metrics.accuracy(
                labels=labels, predictions=tf.argmax(input=logits, axis=1))
    }
    return tf_estimator.EstimatorSpec(
        mode=mode, loss=scalar_loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  logging.set_verbosity(logging.INFO)
  if FLAGS.dpsgd and FLAGS.batch_size % FLAGS.microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

  # Instantiate the tf.Estimator.
  mnist_classifier = tf_estimator.Estimator(
      model_fn=cnn_model_fn, model_dir=FLAGS.model_dir)

  # Training loop.
  steps_per_epoch = 60000 // FLAGS.batch_size
  for epoch in range(1, FLAGS.epochs + 1):
    start_time = time.time()
    # Train the model for one epoch.
    mnist_classifier.train(
        input_fn=make_input_fn('train', FLAGS.batch_size),
        steps=steps_per_epoch)
    end_time = time.time()
    logging.info('Epoch %d time in seconds: %.2f', epoch, end_time - start_time)

    # Evaluate the model and print results
    eval_results = mnist_classifier.evaluate(
        input_fn=make_input_fn('test', FLAGS.batch_size, 1))
    test_accuracy = eval_results['accuracy']
    print('Test accuracy after %d epochs is: %.3f' % (epoch, test_accuracy))

    # Compute the privacy budget expended.
    if FLAGS.dpsgd:
      if FLAGS.noise_multiplier > 0.0:
        eps, _ = compute_dp_sgd_privacy_lib.compute_dp_sgd_privacy(
            60000, FLAGS.batch_size, FLAGS.noise_multiplier, epoch, 1e-5)
        print('For delta=1e-5, the current epsilon is: %.2f' % eps)
      else:
        print('Trained with DP-SGD but with zero noise.')
    else:
      print('Trained with vanilla non-private SGD optimizer')


if __name__ == '__main__':
  tf.app.run(main)

I0826 02:48:25.114225 140383648417664 estimator.py:1843] Using default config.
W0826 02:48:25.117638 140383648417664 estimator.py:1865] Using temporary folder as model directory: /tmp/tmpbf6m5zbk
I0826 02:48:25.121737 140383648417664 estimator.py:202] Using config: {'_model_dir': '/tmp/tmpbf6m5zbk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluste

I0826 02:48:26.009932 140383648417664 dataset_builder.py:456] Dataset mnist is hosted on GCS. It will automatically be downloaded to your
local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

I0826 02:48:27.227159 140383648417664 dataset_info.py:439] Load dataset info from /root/tensorflow_datasets/mnist/3.0.1.incomplete3CTLY1
I0826 02:48:27.233675 140383648417664 dataset_info.py:494] Field info.citation from disk and from code do not match. Keeping the one from code.
I0826 02:48:27.236944 140383648417664 dataset_info.py:494] Field info.splits from disk and from code do not match. Keeping the one from code.
I0826 02:48:27.238362 140383648417664 dataset_info.py:494] Field info.supervised_keys from disk and from code do not match. Keeping the one from code.
I0826 02:48:27.239898 140383648417664 dataset_info.py:494] Field info.module_name from disk and from code do not match. Keeping the one from code.
I0826 02:48:27.241490 140383648417664 dataset_info.py:494] Field info.file_format from disk and from code do not match. Keeping the one from code.
I0826 02:48:27.243322 140383648417664 logging_logger.py:45] Constructing tf.data.Dataset mnist for split train, from /root/tensorflo

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


I0826 02:48:27.470338 140383648417664 estimator.py:1173] Calling model_fn.
I0826 02:48:27.958927 140383648417664 estimator.py:1175] Done calling model_fn.
I0826 02:48:27.968207 140383648417664 basic_session_run_hooks.py:558] Create CheckpointSaverHook.
I0826 02:48:28.524997 140383648417664 monitored_session.py:243] Graph was finalized.
I0826 02:48:28.622129 140383648417664 session_manager.py:527] Running local_init_op.
I0826 02:48:28.638432 140383648417664 session_manager.py:530] Done running local_init_op.
I0826 02:48:29.096368 140383648417664 basic_session_run_hooks.py:629] Calling checkpoint listeners before saving checkpoint 0...
I0826 02:48:29.103442 140383648417664 basic_session_run_hooks.py:633] Saving checkpoints for 0 into /tmp/tmpbf6m5zbk/model.ckpt.
I0826 02:48:29.199784 140383648417664 basic_session_run_hooks.py:641] Calling checkpoint listeners after saving checkpoint 0...
I0826 02:48:59.613519 140383648417664 basic_session_run_hooks.py:266] loss = 2.3040318, step = 0
I082

Test accuracy after 1 epochs is: 0.754
DP-SGD with sampling rate = 0.427% and noise_multiplier = 1.1 iterated over 235 steps satisfies differential privacy with eps = 0.741 and delta = 1e-05.
The optimal RDP order is 13.0.
For delta=1e-5, the current epsilon is: 0.74


I0826 03:38:54.402176 140383648417664 estimator.py:1175] Done calling model_fn.
I0826 03:38:54.405940 140383648417664 basic_session_run_hooks.py:558] Create CheckpointSaverHook.
I0826 03:38:54.474925 140383648417664 monitored_session.py:243] Graph was finalized.
I0826 03:38:54.480037 140383648417664 saver.py:1395] Restoring parameters from /tmp/tmpbf6m5zbk/model.ckpt-234
W0826 03:38:54.517045 140383648417664 deprecation.py:343] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/training/saver.py:1161: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.
I0826 03:38:54.538460 140383648417664 session_manager.py:527] Running local_init_op.
I0826 03:38:54.552843 140383648417664 session_manager.py:530] Done running local_init_op.
I0826 03:38:54.802468 140383648417664 basic_session_run_hooks.py:629] Calling checkpoint listeners befor

Test accuracy after 2 epochs is: 0.859
DP-SGD with sampling rate = 0.427% and noise_multiplier = 1.1 iterated over 469 steps satisfies differential privacy with eps = 0.807 and delta = 1e-05.
The optimal RDP order is 12.0.
For delta=1e-5, the current epsilon is: 0.81


I0826 04:28:31.828010 140383648417664 estimator.py:1175] Done calling model_fn.
I0826 04:28:31.831530 140383648417664 basic_session_run_hooks.py:558] Create CheckpointSaverHook.
I0826 04:28:31.901677 140383648417664 monitored_session.py:243] Graph was finalized.
I0826 04:28:31.904545 140383648417664 saver.py:1395] Restoring parameters from /tmp/tmpbf6m5zbk/model.ckpt-468
I0826 04:28:31.956149 140383648417664 session_manager.py:527] Running local_init_op.
I0826 04:28:31.970744 140383648417664 session_manager.py:530] Done running local_init_op.
I0826 04:28:32.214535 140383648417664 basic_session_run_hooks.py:629] Calling checkpoint listeners before saving checkpoint 468...
I0826 04:28:32.216904 140383648417664 basic_session_run_hooks.py:633] Saving checkpoints for 468 into /tmp/tmpbf6m5zbk/model.ckpt.
I0826 04:28:32.285164 140383648417664 basic_session_run_hooks.py:641] Calling checkpoint listeners after saving checkpoint 468...
I0826 04:28:49.557504 140383648417664 basic_session_run_hoo